In [1]:
import numpy as np
import pandas as pd

In [60]:
def lin_act(x):
    return x

def sigmoid(x):
    return np.exp(x) / (np.exp(x)+1)

def matrix(arr, shape1, shape2):
    return np.array(arr).reshape(shape1, shape2)

del Layer
class Layer:
    def __init__(self, weights, bias, activation=sigmoid):
        self.weights = weights
        self.bias = bias
        self.activation = activation
    
    def apply(self, inputs):
        return self.activation(inputs @ self.weights + self.bias)
    
    
class NN:
    def __init__(self, *layers):
        self.layers = [*layers]
    
    def add_layer(self, layer):
        self.layers.append(layer)
        
    def apply(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer.apply(x)
        return x

## Simple squares

In [54]:
square_simple_training =  pd.read_csv("LAB1_DANE/regression/square-simple-training.csv")
x = np.array(square_simple_training.iloc[:, 1]).reshape(100, 1)
y = np.array(square_simple_training.iloc[:, 2]).reshape(100, 1)

In [63]:
l1 = Layer(matrix([1,2,3,4,5], 1, 5), matrix([1,2,3,4,5], 1, 5))
l2 = Layer(matrix([1,2,3,4,5], 5, 1), matrix([1], 1, 1), activation=lin_act)
l2.apply(l1.apply(x))


array([[14.9264159 ],
       [15.28090132],
       [ 4.22746267],
       [15.94173525],
       [15.26111892],
       [15.93812203],
       [15.91917014],
       [15.77013032],
       [15.94116858],
       [15.74561757],
       [14.45502101],
       [14.61359541],
       [15.06681122],
       [15.87809865],
       [ 4.27077011],
       [15.85842622],
       [15.80653602],
       [ 9.84745587],
       [13.118811  ],
       [15.55890902],
       [15.80506781],
       [15.94372982],
       [15.86468702],
       [15.66865029],
       [15.90617916],
       [10.6595048 ],
       [13.36747664],
       [15.8977938 ],
       [15.81987659],
       [12.54395178],
       [ 4.3901244 ],
       [ 8.38556816],
       [11.73286272],
       [15.45680353],
       [10.99341458],
       [15.83941634],
       [ 3.48493697],
       [14.89279215],
       [15.55889576],
       [ 3.35090547],
       [15.69365942],
       [ 9.22122555],
       [14.74248698],
       [15.77617122],
       [15.87355547],
       [15

In [22]:
nn5 = NN(
    Layer(np.array([1,2,3,4,5]).reshape(1, 5), np.array([0,0,0,0,0]).reshape(1, 5) ),
    Layer(np.array([1,2,3,4,5]).reshape(5, 1), np.array([0,0,0,0,0]).reshape(5, 1) )
)
nn5.apply(x)

(100, 5)
(1, 5)
(100, 1)
(5, 1)


ValueError: operands could not be broadcast together with shapes (100,1) (5,1) 

In [34]:
np.array([1,2,3,4,5]).reshape(5, 1)

array([[1],
       [2],
       [3],
       [4],
       [5]])

In [50]:
l1_w = np.array([1,2,3,4,5]).reshape(1,5)
x1 = x @ l1_w
l2_w = np.array([1,2,3,4,5]).reshape(5,1)
x1 @ l2_w

array([[ -9.4348462 ],
       [  1.38603011],
       [-75.29452617],
       [104.9064334 ],
       [  0.61211541],
       [101.83349485],
       [ 88.39281931],
       [ 40.69591312],
       [104.41107249],
       [ 36.62635491],
       [-18.31526183],
       [-15.75096222],
       [ -5.7915583 ],
       [ 68.55335932],
       [-75.00472546],
       [ 61.64311184],
       [ 47.87578981],
       [-49.5316164 ],
       [-32.24056696],
       [ 16.52448992],
       [ 47.55440034],
       [106.6923106 ],
       [ 63.7122715 ],
       [ 26.54902164],
       [ 81.06527082],
       [-46.02626743],
       [-30.27732667],
       [ 76.92050841],
       [ 50.94192792],
       [-36.19513762],
       [-74.22465677],
       [-55.45777438],
       [-40.8458076 ],
       [  9.78422361],
       [-44.49850911],
       [ 55.97599875],
       [-80.98218779],
       [-10.21818181],
       [ 16.52348385],
       [-82.19766831],
       [ 29.45877048],
       [-52.10324743],
       [-13.38705112],
       [ 41